##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Treinamento e serviço Vertex AI com TFX e Vertex Pipelines


<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
<td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/gcp/vertex_pipelines_vertex_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Executar no Google Cloud Vertex AI Workbench</a></td>
</table></div>


Este tutorial baseado em notebook criará e executará um pipeline TFX que treina um modelo de ML usando o serviço Vertex AI Training e o publica na Vertex AI para disponibilização do serviço.

Este notebook é baseado no pipeline TFX que construímos no [Tutorial Pipeline TFX simples para Vertex Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple). Se você ainda não leu esse tutorial, leia-o antes de prosseguir com este notebook.

Você pode treinar modelos na Vertex AI usando AutoML ou usar treinamento personalizado. No treinamento personalizado, você pode selecionar vários tipos de máquinas diferentes para potencializar seus trabalhos de treinamento, permitir treinamento distribuído, usar tunagem de hiperparâmetros e acelerar com GPUs.

Você também pode servir solicitações de previsão implantando o modelo treinado em modelos Vertex AI e criando um endpoint.

Neste tutorial, usaremos o Vertex AI Training com trabalhos personalizados para treinar um modelo em um pipeline do TFX. Também implantaremos o modelo para atender solicitações de previsão usando o Vertex AI.


Este notebook foi criado para ser executado no [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) ou em [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks). Se você não estiver usando um deles, basta clicar no botão "Executar no Google Colab" acima.

## Configuração

Se você concluiu o [Tutorial pipeline TFX simples para Vertex Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple), você terá um projeto GCP funcional e um bucket GCS e isso é tudo que precisamos para este tutorial. Por favor, leia esse tutorial primeiro se você já não o fez.

### Instale os pacotes Python

Instalaremos os pacotes Python necessários, incluindo TFX e KFP, para criar pipelines de ML e enviar trabalhos para o Vertex Pipelines.

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"

### Desinstale o shapely

TODO(b/263441833) Esta é uma solução temporária para evitar um ImportError. Em última análise, isto deverá ser resolvido com suporte a uma versão mais recente do Bigquery, em vez de desinstalar outras dependências extras.

In [ ]:
!pip uninstall shapely -y

#### Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime clicando no botão "RESTART RUNTIME" acima ou usando o menu "Runtime &gt; Restart runtime ...". Isso é necessário devido à maneira como o Colab carrega os pacotes.

Se você não estiver no Colab, poderá reiniciar o runtime com a seguinte célula.

In [ ]:
# docs_infra: no_execute
import sys
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

### Faça login no Google para este notebook

Se você estiver executando este notebook no Colab, autentique-se com sua conta de usuário:

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

**Se você estiver no AI Platform Notebooks**, autentique-se no Google Cloud antes de executar a próxima seção, executando

```sh
gcloud auth login
```

**na janela do Terminal** (que você pode abrir em **File** &gt; **New** no menu). Você só precisa fazer isso uma vez por instância de notebook.

Verifique as versões dos pacotes.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

### Configuração de variáveis

Definiremos algumas variáveis ​​usadas para personalizar os pipelines abaixo. As seguintes informações são necessárias:

- ID do projeto GCP. Consulte [Identificando o ID do seu projeto](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects).
- Região do GCP para executar pipelines. Para obter mais informações sobre as regiões em que o Vertex Pipelines está disponível, veja o [guia de locais do Vertex AI](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability).
- Bucket do Google Cloud Storage para armazenar saídas de pipeline.

**Insira os valores necessários na célula abaixo antes de executá-la**.


In [ ]:
GOOGLE_CLOUD_PROJECT = ''     # <--- ENTER THIS
GOOGLE_CLOUD_REGION = ''      # <--- ENTER THIS
GCS_BUCKET_NAME = ''          # <--- ENTER THIS

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

Defina o `gcloud` para usar seu projeto.

In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

In [ ]:
PIPELINE_NAME = 'penguin-vertex-training'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Name of Vertex AI Endpoint.
ENDPOINT_NAME = 'prediction-' + PIPELINE_NAME

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

### Preparação dos dados de exemplo

Usaremos o mesmo [dataset Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) do [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

Existem quatro características numéricas neste dataset que já foram normalizados para ter intervalo [0,1]. Construiremos um modelo de classificação que prevê as `species` de pinguins.

Precisamos fazer nossa própria cópia do dataset. Como o TFX ExampleGen lê entradas de um diretório, precisamos criar um diretório e copiar o dataset para ele no GCS.

In [ ]:
!gsutil cp gs://download.tensorflow.org/data/palmer_penguins/penguins_processed.csv {DATA_ROOT}/

Dê uma olhada rápida no arquivo CSV.

In [ ]:
!gsutil cat {DATA_ROOT}/penguins_processed.csv | head

## Crie um pipeline

Nosso pipeline será muito semelhante ao pipeline que criamos no [Tutorial pipeline TFX simples para Vertex Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple). O pipeline consistirá em três componentes, CsvExampleGen, Trainer e Pusher. Mas usaremos um componente Trainer e Pusher especial. O componente Trainer moverá as cargas de trabalho de treinamento para a Vertex AI, e o componente Pusher publicará o modelo de ML treinado na Vertex AI em vez de num sistema de arquivos.

O TFX fornece um `Trainer` especial para enviar trabalhos de treinamento ao serviço Vertex AI Training. Tudo o que precisamos fazer é usar `Trainer` no módulo de extensão em vez do componente `Trainer` padrão junto com alguns parâmetros necessários do GCP.

Neste tutorial, executaremos jobs do Vertex AI Training inicialmente usando apenas CPUs e depois com uma GPU.

O TFX também fornece um `Pusher` especial para fazer upload do modelo para *Vertex AI Models*. O `Pusher` também criará o recurso *Vertex AI Endpoint* para atender previsões on-line. Veja a [documentação do Vertex AI](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions) para saber mais sobre as previsões on-line fornecidas pela Vertex AI.

### Escreva o código do modelo.

O modelo em si é muito parecido com o modelo do [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

Adicionaremos a função `_get_distribution_strategy()` que cria uma [estratégia de distribuição TensorFlow](https://www.tensorflow.org/guide/distributed_training) e é usada em `run_fn` para usar MirroredStrategy se a GPU estiver disponível.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple and
# slightly modified run_fn() to add distribution_strategy.

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
        for feature in _FEATURE_KEYS
    }, _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# NEW: Read `use_gpu` from the custom_config of the Trainer.
#      if it uses GPU, enable MirroredStrategy.
def _get_distribution_strategy(fn_args: tfx.components.FnArgs):
  if fn_args.custom_config.get('use_gpu', False):
    logging.info('Using MirroredStrategy with one GPU.')
    return tf.distribute.MirroredStrategy(devices=['device:GPU:0'])
  return None


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  # NEW: If we have a distribution strategy, build a model in a strategy scope.
  strategy = _get_distribution_strategy(fn_args)
  if strategy is None:
    model = _make_keras_model()
  else:
    with strategy.scope():
      model = _make_keras_model()

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Copie o arquivo do módulo para o GCS, que pode ser acessado nos componentes do pipeline.

Caso contrário, você talvez queira criar uma imagem de container incluindo o arquivo do módulo e usar a imagem para executar o pipeline e os jobs do AI Platform Training.

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

### Escreva uma definição de pipeline

Definiremos uma função para criar um pipeline TFX. Ela terá os mesmos três componentes do [Tutorial pipeline TFX simples](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple), mas usamos um componente `Trainer` e `Pusher` no módulo de extensão GCP.

O `tfx.extensions.google_cloud_ai_platform.Trainer` se comporta como um `Trainer` normal, mas apenas move o cálculo do treinamento do modelo para a nuvem. Ele inicia um job personalizado no serviço Vertex AI Training e o componente de treinamento no sistema de orquestração irá apenas aguardar até que o trabalho do Vertex AI Training seja concluído.

O `tfx.extensions.google_cloud_ai_platform.Pusher` cria um modelo Vertex AI e um endpoint Vertex AI usando o modelo treinado.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, endpoint_name: str, project_id: str,
                     region: str, use_gpu: bool) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Configuration for Vertex AI Training.
  # This dictionary will be passed as `CustomJobSpec`.
  vertex_job_spec = {
      'project': project_id,
      'worker_pool_specs': [{
          'machine_spec': {
              'machine_type': 'n1-standard-4',
          },
          'replica_count': 1,
          'container_spec': {
              'image_uri': 'gcr.io/tfx-oss-public/tfx:{}'.format(tfx.__version__),
          },
      }],
  }
  if use_gpu:
    # See https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec#acceleratortype
    # for available machine types.
    vertex_job_spec['worker_pool_specs'][0]['machine_spec'].update({
        'accelerator_type': 'NVIDIA_TESLA_K80',
        'accelerator_count': 1
    })

  # Trains a model using Vertex AI Training.
  # NEW: We need to specify a Trainer for GCP with related configs.
  trainer = tfx.extensions.google_cloud_ai_platform.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5),
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.TRAINING_ARGS_KEY:
              vertex_job_spec,
          'use_gpu':
              use_gpu,
      })

  # NEW: Configuration for pusher.
  vertex_serving_spec = {
      'project_id': project_id,
      'endpoint_name': endpoint_name,
      # Remaining argument is passed to aiplatform.Model.deploy()
      # See https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#deploy_the_model
      # for the detail.
      #
      # Machine type is the compute resource to serve prediction requests.
      # See https://cloud.google.com/vertex-ai/docs/predictions/configure-compute#machine-types
      # for available machine types and acccerators.
      'machine_type': 'n1-standard-4',
  }

  # Vertex AI provides pre-built containers with various configurations for
  # serving.
  # See https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
  # for available container images.
  serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest'
  if use_gpu:
    vertex_serving_spec.update({
        'accelerator_type': 'NVIDIA_TESLA_K80',
        'accelerator_count': 1
    })
    serving_image = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-6:latest'

  # NEW: Pushes the model to Vertex AI.
  pusher = tfx.extensions.google_cloud_ai_platform.Pusher(
      model=trainer.outputs['model'],
      custom_config={
          tfx.extensions.google_cloud_ai_platform.ENABLE_VERTEX_KEY:
              True,
          tfx.extensions.google_cloud_ai_platform.VERTEX_REGION_KEY:
              region,
          tfx.extensions.google_cloud_ai_platform.VERTEX_CONTAINER_IMAGE_URI_KEY:
              serving_image,
          tfx.extensions.google_cloud_ai_platform.SERVING_ARGS_KEY:
            vertex_serving_spec,
      })

  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

## Execute o pipeline no Vertex Pipelines.

Usaremos Vertex Pipelines para executar o pipeline, como fizemos no [Tutorial Pipeline TFX simples para Vertex Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple).

In [ ]:
# docs_infra: no_execute
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        # We will use CPUs only for now.
        use_gpu=False))

O arquivo de definição gerado pode ser enviado usando o cliente Google Cloud aiplatform no pacote `google-cloud-aiplatform`.

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

Agora você pode visitar o link na saída acima ou visitar 'Vertex AI &gt; Pipelines' no [Console do Google Cloud](https://console.cloud.google.com/) para acompanhar o andamento.

## Teste com uma solicitação de previsão

Assim que o pipeline for concluído, você encontrará um modelo *implantado* em um dos endpoints em 'Vertex AI &gt; Endpoints'. Precisamos saber o ID do endpoint para enviar uma solicitação de previsão ao novo endpoint. Isto é diferente do *nome do endpoint* que inserimos acima. Você pode encontrar o ID na [página Endpoints](https://console.cloud.google.com/vertex-ai/endpoints) no `Google Cloud Console`. É um número bem longo.

**Defina o ENDPOINT_ID abaixo antes de executá-lo.**


In [ ]:
ENDPOINT_ID=''     # <--- ENTER THIS
if not ENDPOINT_ID:
    from absl import logging
    logging.error('Please set the endpoint id.')

Usamos o mesmo cliente aiplatform para enviar uma solicitação ao endpoint. Enviaremos uma solicitação de previsão para classificação de espécies de pinguins. A entrada são as quatro características que usamos, e o modelo retornará três valores, porque nosso modelo gera um valor para cada espécie.

Por exemplo, o exemplo específico a seguir tem o maior valor no índice '2' e imprimirá '2'.


In [ ]:
# docs_infra: no_execute
import numpy as np

# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

# Set data values for the prediction request.
# Our model expects 4 feature inputs and produces 3 output values for each
# species. Note that the output is logit value rather than probabilities.
# See the model code to understand input / output structure.
instances = [{
    'culmen_length_mm':[0.71],
    'culmen_depth_mm':[0.38],
    'flipper_length_mm':[0.98],
    'body_mass_g': [0.78],
}]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
# Send a prediction request and get response.
response = client.predict(endpoint=endpoint, instances=instances)

# Uses argmax to find the index of the maximum value.
print('species:', np.argmax(response.predictions[0]))

Para obter informações detalhadas sobre a previsão on-line, visite a [página Endpoints](https://console.cloud.google.com/vertex-ai/endpoints) no `Google Cloud Console`, onde você encontrará um guia sobre como enviar exemplos de solicitações e links para mais características.

## Execute o pipeline usando uma GPU


A Vertex AI oferece suporte ao treinamento usando vários tipos de máquinas, incluindo suporte para GPUs. Consulte a [ referência de especificações de máquina](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec#acceleratortype) para obter as opções disponíveis.

Já definimos nosso pipeline para dar suporte ao treinamento usando GPU. Tudo o que precisamos fazer é definir o sinalizador `use_gpu` como True. Em seguida, um pipeline será criado com uma especificação de máquina incluindo NVIDIA_TESLA_K80 e nosso código de treinamento de modelo usará `tf.distribute.MirroredStrategy`.

Observe que o sinalizador `use_gpu` não faz parte da API Vertex ou TFX. Ele é usado apenas para controlar o código de treinamento neste tutorial.

In [ ]:
# docs_infra: no_execute
runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        endpoint_name=ENDPOINT_NAME,
        project_id=GOOGLE_CLOUD_PROJECT,
        region=GOOGLE_CLOUD_REGION,
        # Updated: Use GPUs. We will use a NVIDIA_TESLA_K80 and 
        # the model code will use tf.distribute.MirroredStrategy.
        use_gpu=True))

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

Agora você pode visitar o link na saída acima ou visitar 'Vertex AI &gt; Pipelines' no [Console do Google Cloud](https://console.cloud.google.com/) para acompanhar o andamento.

## Limpeza

Você criou um modelo e um endpoint Vertex AI neste tutorial. Exclua esses recursos para evitar cobranças indesejadas, acessando [Endpoints](https://console.cloud.google.com/vertex-ai/endpoints) e fazendo *undeploy* do modelo do endpoint primeiro. Depois você pode excluir o endpoint e o modelo separadamente.